In [ ]:
# default_exp esm

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import torch
import esm
import pandas as pd

In [ ]:
# export
from berteome import berteomeDF

In [ ]:
# export
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()

In [ ]:
# export
batch_converter = alphabet.get_batch_converter()

In [ ]:
# export
def naturalAAIndex(aas):
    
    model_alphabet=alphabet.all_toks
    naturalAAIndices = []
    for aa in aas:
      naturalAAIndices.append(model_alphabet.index(aa))
    return naturalAAIndices

In [ ]:
assert naturalAAIndex("ACDEFGHIKLMNPQRSTVWY") == [5, 23, 13, 9, 18, 6, 21, 12, 15, 4, 20, 17, 14, 16, 10, 8, 11, 7, 22, 19]

In [ ]:
# export
def maskifySeq(seq, i):
    seqList = list(seq)
    seqList[i] = "<mask>"
    return seqList

In [ ]:
# export
def getNatProbs(natAAList,probList):
    natProbList = []
    for natAAIndex in natAAList:
      natProbList.append(probList[natAAIndex])
    return natProbList

In [ ]:
# export
def esmPredictionDF(seq, aas="ACDEFGHIKLMNPQRSTVWY"): # this should probably be a class where model/alphabet are declared??
    naturalAAIndices=naturalAAIndex(aas)
    esmPredDict = {}
    for wtIndex in range(len(seq)):
      maskSeq = maskifySeq(seq, wtIndex) # import from other module, ie seqtools.maskifySeq(), or not, maybe both classes should just be in the same module...
      data = [("protein1", maskSeq)]
      batch_labels, batch_strs, batch_tokens = batch_converter(data)
      with torch.no_grad():
        results = model(batch_tokens)
        logits = results['logits']
        prob = torch.softmax(logits,dim=2)
      esmPredDict[wtIndex] = [i.item() for i in getNatProbs(naturalAAIndices, prob[0, wtIndex +1])]
    esmPredDF = berteomeDF.modelPredDF(esmPredDict,seq, aas).predDf
    
    return esmPredDF

In [ ]:
esmPredictionDF("MENDEL")

,wt,wtIndex,A,C,D,E,F,G,H,I,...,M,N,P,Q,R,S,T,V,W,Y
0,M,1,0.033952,0.007039,0.054737,0.063365,0.018590,0.029267,0.010216,0.028865,...,0.456082,0.029166,0.023344,0.019043,0.023603,0.030170,0.023352,0.033982,0.004791,0.013076
1,E,2,0.058999,0.021031,0.054622,0.072057,0.037186,0.053090,0.025848,0.065879,...,0.024329,0.059710,0.035728,0.040369,0.052300,0.073618,0.057319,0.062975,0.014029,0.030339
2,N,3,0.055626,0.014060,0.077887,0.122153,0.034727,0.058822,0.019171,0.061401,...,0.031318,0.044172,0.029668,0.034335,0.052372,0.057394,0.047432,0.069432,0.013282,0.025859
3,D,4,0.044610,0.017627,0.036298,0.071025,0.031629,0.049782,0.023518,0.061833,...,0.058515,0.043762,0.037227,0.054164,0.055120,0.060091,0.052912,0.070125,0.017567,0.027211
4,E,5,0.045966,0.027927,0.047431,0.057574,0.052450,0.054545,0.030670,0.071293,...,0.025417,0.051106,0.033450,0.038940,0.055194,0.076423,0.050060,0.063550,0.017663,0.038091
5,L,6,0.048226,0.016231,0.060284,0.100391,0.031534,0.052584,0.023015,0.065281,...,0.029354,0.063840,0.025546,0.039313,0.062937,0.064147,0.053790,0.060177,0.013714,0.028147


In [ ]:
mendel

,wt,wtIndex,A,C,D,E,F,G,H,I,...,M,N,P,Q,R,S,T,V,W,Y
0,M,1,0.033952,0.007039,0.054737,0.063365,0.018590,0.029267,0.010216,0.028865,...,0.456082,0.029166,0.023344,0.019043,0.023603,0.030170,0.023352,0.033982,0.004791,0.013076
1,E,2,0.058999,0.021031,0.054622,0.072057,0.037186,0.053090,0.025848,0.065879,...,0.024329,0.059710,0.035728,0.040369,0.052300,0.073618,0.057319,0.062975,0.014029,0.030339
2,N,3,0.055626,0.014060,0.077887,0.122153,0.034727,0.058822,0.019171,0.061401,...,0.031318,0.044172,0.029668,0.034335,0.052372,0.057394,0.047432,0.069432,0.013282,0.025859
3,D,4,0.044610,0.017627,0.036298,0.071025,0.031629,0.049782,0.023518,0.061833,...,0.058515,0.043762,0.037227,0.054164,0.055120,0.060091,0.052912,0.070125,0.017567,0.027211
4,E,5,0.045966,0.027927,0.047431,0.057574,0.052450,0.054545,0.030670,0.071293,...,0.025417,0.051106,0.033450,0.038940,0.055194,0.076423,0.050060,0.063550,0.017663,0.038091
5,L,6,0.048226,0.016231,0.060284,0.100391,0.031534,0.052584,0.023015,0.065281,...,0.029354,0.063840,0.025546,0.039313,0.062937,0.064147,0.053790,0.060177,0.013714,0.028147
